# Network

In [31]:
import torch
import os
import re
import pickle
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import argparse as ap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from matplotlib import pyplot as plt

In [32]:
DEFAULT_MODEL_PATH = "..\\models"
DEFAULT_DATA_PATH = "..\\csv\\data.csv"
GENERIC_MODEL_FILE_NAME = "model_epoch_"
GENERIC_SCALAR_FILE_NAME = "scalar_epoch_"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
modelRegexFilter = re.compile(GENERIC_MODEL_FILE_NAME + r"(?P<epoch>\d+)")
scalerRegexFilter = re.compile(GENERIC_SCALAR_FILE_NAME + r"(?P<epoch>\d+)")

class Net(nn.Module):
    def __init__(self, input_size, num_classes):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 16)
        self.fc6 = nn.Linear(16, num_classes)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = torch.relu(self.fc5(x))
        x = self.fc6(x)
        return x
    
class CustomDataset(Dataset):
    def __init__(self, features, labels):
        self.features = torch.tensor(features, dtype=torch.float32).to(device)
        self.labels = torch.tensor(labels.values, dtype=torch.float32).to(device)
        self.inputs = torch.tensor(features, dtype=torch.float32).to(device)   
        self.inputs = torch.tensor(features, dtype=torch.float32).to(device) 
          
    def __len__(self):
        return len(self.features)
    
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

def saveModel(model, scaler, path, epoch, fileLimit):
    if not os.path.exists(path):
        os.makedirs(path)

    torch.save(model.state_dict(), path + "\\"+ GENERIC_MODEL_FILE_NAME + str(epoch) + ".pt")
    with open(path+ "\\" + GENERIC_SCALAR_FILE_NAME+str(epoch) + ".pkl", "wb") as file:
        pickle.dump(scaler, file)
    
    files = os.listdir(path)
    files = [f for f in files if os.path.isfile(path+'/'+f)]
    if len(files) > 0:
        modelEpochList = [int(modelRegexFilter.match(file).groups("epoch")[0])  for file in files if modelRegexFilter.match(file)]
        scalarEpochList = [int(scalerRegexFilter.match(file).groups("epoch")[0])  for file in files if scalerRegexFilter.match(file)]
        modelEpochList.sort(reverse=True)
        scalarEpochList.sort(reverse=True)
        for epoch in modelEpochList[fileLimit:]:
            os.remove(path + "\\"+ GENERIC_MODEL_FILE_NAME + str(epoch) + ".pt")
            os.remove(path+ "\\" + GENERIC_SCALAR_FILE_NAME+str(epoch) + ".pkl")



def loadModel(path, input_size, num_classes):
    model = Net(input_size, num_classes)
    scaler = StandardScaler()
    epoch = 0
    if os.path.exists(path):
        files = os.listdir(path)
        files = [f for f in files if os.path.isfile(path+'/'+f)]
        if len(files) > 0:
            modelEpochList = [int(modelRegexFilter.match(file).groups("epoch")[0])  for file in files if modelRegexFilter.match(file)]
            scalarEpochList = [int(scalerRegexFilter.match(file).groups("epoch")[0])  for file in files if scalerRegexFilter.match(file)]
            modelEpochList.sort(reverse=True)
            scalarEpochList.sort(reverse=True)
            if modelEpochList[0] != scalarEpochList[0]:
                for e in modelEpochList:
                    if e in scalarEpochList:
                        epoch = e
            else:
                epoch = modelEpochList[0]

            
            model.load_state_dict(torch.load(path +"\\"+ GENERIC_MODEL_FILE_NAME + str(epoch) + ".pt"))
            with open(path+ "\\" + GENERIC_SCALAR_FILE_NAME+str(epoch) + ".pkl", "rb") as file:
                scaler = pickle.load(file)

    return model.to(device), scaler, epoch

def readCSV(dataPath):
    if os.path.exists(dataPath):
        files = os.listdir(dataPath)
        df = pd.DataFrame()
        for file in files:
            dfTemp = pd.read_csv(dataPath+file, sep=",")
            df = pd.concat([df,dfTemp],ignore_index=True)
            # print(dfTemp)
        # print(df)
        return df

cuda


In [33]:
from tqdm.auto import tqdm
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np


modelName = "INSERT_MODEL_NAME_HERE"
dataPath = '..\\csv\\INSERT_CSV_PATH_HERE\\'
# dataPath = args.data_path
DEFAULT_MODEL_PATH = "..\\models"
modelPath = DEFAULT_MODEL_PATH
amountOfEpochs = 10
epochs = 200
fileLimit = 5

# Accuracy calculation
def calculate_accuracy(true_labels, predictions, threshold=0.1):
    correct = np.abs(true_labels - predictions) < threshold
    accuracy = correct.mean() * 100
    return accuracy

# selectedFeatures = ['speedX', 'speedY','angle','trackPos', 'brake', 'accel', 'steer']
selectedFeatures = [' SpeedX', ' SpeedY', 'Angle', 'TrackPosition', 'Braking', ' Acceleration', 'Steering', 
                    ' Track_1','Track_2','Track_3','Track_4','Track_5','Track_6','Track_7','Track_8','Track_9','Track_10','Track_11','Track_12','Track_13','Track_14','Track_15','Track_16','Track_17','Track_18','Track_19',
                    ' SpeedZ', 
                    ' Opponent_1', 'Opponent_2', 'Opponent_3', 'Opponent_4', 'Opponent_5', 'Opponent_6', 'Opponent_7', 'Opponent_8', 'Opponent_9', 'Opponent_10',
                    'Opponent_11', 'Opponent_12', 'Opponent_13', 'Opponent_14', 'Opponent_15', 'Opponent_16', 'Opponent_17', 'Opponent_18', 'Opponent_19', 'Opponent_20',
                    'Opponent_21', 'Opponent_22', 'Opponent_23', 'Opponent_24', 'Opponent_25', 'Opponent_26', 'Opponent_27', 'Opponent_28', 'Opponent_29', 'Opponent_30',
                    'Opponent_31', 'Opponent_32', 'Opponent_33', 'Opponent_34', 'Opponent_35', 'Opponent_36']

# Load data
df = readCSV(dataPath)
# print(df)
print(df.head())

dfSelected = df.copy()
dfSelected.dropna(axis=0)
input = dfSelected[[' SpeedX', ' SpeedY', 'Angle', 'TrackPosition', 
                    ' Track_1','Track_2','Track_3','Track_4','Track_5','Track_6','Track_7','Track_8','Track_9','Track_10','Track_11','Track_12','Track_13','Track_14','Track_15','Track_16','Track_17','Track_18','Track_19' 
                    ,' SpeedZ',' Opponent_1', 'Opponent_2', 'Opponent_3', 'Opponent_4', 'Opponent_5', 'Opponent_6', 'Opponent_7', 'Opponent_8', 'Opponent_9', 'Opponent_10',
                    'Opponent_11', 'Opponent_12', 'Opponent_13', 'Opponent_14', 'Opponent_15', 'Opponent_16', 'Opponent_17', 'Opponent_18', 'Opponent_19', 'Opponent_20',
                    'Opponent_21', 'Opponent_22', 'Opponent_23', 'Opponent_24', 'Opponent_25', 'Opponent_26', 'Opponent_27', 'Opponent_28', 'Opponent_29', 'Opponent_30',
                    'Opponent_31', 'Opponent_32', 'Opponent_33', 'Opponent_34', 'Opponent_35', 'Opponent_36']]
                    
output = dfSelected[['Braking', ' Acceleration', 'Steering']]

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(input, output, test_size=0.2)

input_size = X_train.shape[1]
num_classes = len(set(output))

net, scaler, oldEpoch = loadModel(modelPath + "\\" + modelName, input_size, num_classes)

# # Standardize scaler features
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)


train_dataset = CustomDataset(X_train, y_train)
test_dataset = CustomDataset(X_test, y_test)
#dataloader
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

criterion = nn.L1Loss()  # Mean Absolute Error
# optimizer = torch.optim.SGD(net.parameters(), lr=0.1)
optimizer = optim.Adam(net.parameters(), lr=0.001)

# Training loop
loss_data = []
test_losses = []

for epoch in tqdm(range(epochs)):
    if epoch != 0 and epoch % amountOfEpochs == 0:
        saveModel(net, scaler, modelPath + "\\" + modelName, epoch+oldEpoch, fileLimit)
    net.train()
    running_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader):
        # Move inputs and labels to GPU
        inputs, labels = inputs.to(device), labels.to(device)
        # print("inputs", inputs)
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = net(inputs)
        # print("outputs", outputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        # print(loss.item())
        # if i % 100 == 99:    # Print every 100 mini-batches
        #     print(f'Epoch [{epoch+1}/{epochs}], Step [{i+1}/{len(train_loader)}], Loss: {running_loss/100}')
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader)}                    ', end="\r")
    loss_data.append(running_loss/len(train_loader))

    # Evaluation
    net.eval()
    with torch.no_grad():
        test_loss = 0
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss.item()
        test_losses.append(test_loss/len(test_loader))
    
print("")
print(test_losses)

plt.plot(loss_data, label = "training_loss")
plt.plot(test_losses, label = "test_loss")
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss')
plt.legend()
plt.show()
saveModel(net, scaler, modelPath + "\\" + modelName, epoch+oldEpoch, fileLimit)

# Evaluation
# # input = dfSelected[['speedX', 'speedY','angle','trackPos']]
# # input = [20.194000,291.326358,2.366310,0.044153,0.486511,0.000000,1.000033,-0.020732]
# input = [[200, 2.366310, 0.044153, 1]]
# column_names = ['speedX', 'speedY','angle','trackPos']
# input_df = pd.DataFrame(input, columns=column_names)
# input_df = scaler.transform(input_df)
# input_tensor = torch.tensor(input_df, dtype=torch.float32).to(device)
# print("input_tensor", input_tensor)

# Evaluation
net.eval()
with torch.no_grad():
    predictions = []
    true_labels = []
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = net(inputs)
        predictions.extend(outputs.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Convert to numpy arrays
predictions = np.array(predictions)
true_labels = np.array(true_labels)

# Calculate MAE and RMSE
mae = mean_absolute_error(true_labels, predictions)
rmse = np.sqrt(mean_squared_error(true_labels, predictions))

print("Mean Absolute Error:", mae)
print("Root Mean Squared Error:", rmse)

accuracy = calculate_accuracy(true_labels, predictions)
print(f"Accuracy: {accuracy:.2f}%")

          Angle   CurrentLapTime   Damage   DistanceFromStart  \
0 -1.748460e-07           -0.982      0.0             22125.8   
1 -1.748460e-07           -0.962      0.0             22125.8   
2 -1.748460e-07           -0.942      0.0             22125.8   
3 -1.748460e-07           -0.922      0.0             22125.8   
4 -1.748460e-07           -0.902      0.0             22125.8   

    DistanceCovered   FuelLevel   Gear   LastLapTime   Opponent_1  Opponent_2  \
0               0.0     94.0000      0           0.0      30.0009       200.0   
1               0.0     94.0000      0           0.0      30.0009       200.0   
2               0.0     93.9999      0           0.0      30.0009       200.0   
3               0.0     93.9998      0           0.0      30.0009       200.0   
4               0.0     93.9997      0           0.0      30.0009       200.0   

   ...   WheelSpinVelocity_1  WheelSpinVelocity_2  WheelSpinVelocity_3  \
0  ...                   0.0                  0.

  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
# dataPath = '..\\csv\\cg-track2\\'
# readCSV(dataPath)

# df = pd.read_csv(dataPath, sep=",")
